In [82]:
import pandas as pd
import numpy as np

columns = ['datetime', 'city', 'state', 'country', 'shape', 'duration_s', 'hours', 'comments', 'date_p', 'latitude', 'longitude']
df = pd.read_csv("complete.csv", header=0, names=columns, error_bad_lines=False, warn_bad_lines=False, low_memory=False)
df.head()

,datetime,city,state,country,shape,duration_s,hours,comments,date_p,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611


In [83]:
#dropando colunas que não serão utilizadas

df.drop('hours', inplace=True, axis=1)
df.drop('date_p', inplace=True, axis=1)

In [84]:
#transformando colunas que são números do tipo object em numéricas. Os valores que estiver errados, são transformandos em NaN.

df['duration_s'] = df['duration_s'].apply(lambda x: pd.to_numeric(x, errors='coerce'))
df['latitude'] = df['latitude'].apply(lambda x: pd.to_numeric(x, errors='coerce'))
df['longitude'] = df['longitude'].apply(lambda x: pd.to_numeric(x, errors='coerce'))

In [85]:
#separandoo date e time da coluna datetime

df['datetime'] = df['datetime'].apply(lambda x: x.split(' '))

In [86]:
#separando date e time em colunas diferentes, e fazendo um depara em time = 24:00

df['date'] = df['datetime'].apply(lambda x:x[0])
df.drop('datetime', axis=1, inplace=True)
df.head()

,city,state,country,shape,duration_s,comments,latitude,longitude,date
0,san marcos,tx,us,cylinder,2700.0,This event took place in early fall around 194...,29.883056,-97.941111,10/10/1949
1,lackland afb,tx,NaN,light,7200.0,1949 Lackland AFB&#44 TX. Lights racing acros...,29.384210,-98.581082,10/10/1949
2,chester (uk/england),NaN,gb,circle,20.0,Green/Orange circular disc over Chester&#44 En...,53.200000,-2.916667,10/10/1955
3,edna,tx,us,circle,20.0,My older brother and twin sister were leaving ...,28.978333,-96.645833,10/10/1956
4,kaneohe,hi,us,light,900.0,AS a Marine 1st Lt. flying an FJ4B fighter/att...,21.418056,-157.803611,10/10/1960


In [87]:
#transformando as colunas date e time do formato object para datetime

df['date'] =  pd.to_datetime(df['date'], format='%m/%d/%Y')

In [88]:
df.dtypes

city                  object
state                 object
country               object
shape                 object
duration_s           float64
comments              object
latitude             float64
longitude            float64
date          datetime64[ns]
dtype: object

In [89]:
#dropando linhas que possuem algum valor NaN

df.dropna(inplace=True)

In [90]:
#verificando quais os shapes existentes

shapes = df['shape'].groupby(df['shape']).count()
shapes

shape
changed          1
changing      1781
chevron        882
cigar         1826
circle        6960
cone           291
crescent         1
cross          223
cylinder      1144
delta            8
diamond       1056
disk          4772
egg            661
fireball      5590
flare            1
flash         1233
formation     2215
hexagon          1
light        14976
other         5105
oval          3405
pyramid          1
rectangle     1180
round            2
sphere        4744
teardrop       656
triangle      7244
unknown       5311
Name: shape, dtype: int64

In [91]:
#agrupando descrição de formas dos UFOs as quais julgamos serem pertinentes a uma única classe

df['shape'] = df['shape'].replace(to_replace ='changed', value = 'changing')
df['shape'] = df['shape'].replace(to_replace ='other', value = 'unknown')
df['shape'] = df['shape'].replace(to_replace ='delta', value = 'triangle')
df['shape'] = df['shape'].replace(to_replace ='crescent', value = 'unknown')
df['shape'] = df['shape'].replace(to_replace ='round', value = 'circle')
df['shape'] = df['shape'].replace(to_replace ='egg', value = 'oval')
df['shape'] = df['shape'].replace(to_replace ='flare', value = 'light')

In [92]:
#transformando a coluna duration_s de segundos para minutos

df['duration_s'] = df['duration_s'].apply(lambda x: x/60)
df.rename(columns={'duration_s': 'duration_m'}, inplace=True)

In [93]:
df.head()

,city,state,country,shape,duration_m,comments,latitude,longitude,date
0,san marcos,tx,us,cylinder,45.000000,This event took place in early fall around 194...,29.883056,-97.941111,1949-10-10
3,edna,tx,us,circle,0.333333,My older brother and twin sister were leaving ...,28.978333,-96.645833,1956-10-10
4,kaneohe,hi,us,light,15.000000,AS a Marine 1st Lt. flying an FJ4B fighter/att...,21.418056,-157.803611,1960-10-10
5,bristol,tn,us,sphere,5.000000,My father is now 89 my brother 52 the girl wit...,36.595000,-82.188889,1961-10-10
7,norwalk,ct,us,disk,20.000000,A bright orange color changing to reddish colo...,41.117500,-73.408333,1965-10-10


In [94]:
#exportando o df pre processado para um csv, para posterior utilização

df.to_csv('pre_processado_completo.csv', index=None)